In [ ]:
import os
import random
import pandas as pd
import cv2

def random_augmentation(image):
    """Apply a random augmentation to the image."""
    aug_type = random.choice(['flip', 'rotate_5', 'rotate_-5', 'scale_0.8'])
    
    if aug_type == 'flip':
        image = cv2.flip(image, 1)  
    elif aug_type == 'rotate_5':
        M = cv2.getRotationMatrix2D((image.shape[1]//2, image.shape[0]//2), 5, 1)
        image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
    elif aug_type == 'rotate_-5':
        M = cv2.getRotationMatrix2D((image.shape[1]//2, image.shape[0]//2), -5, 1)
        image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
    elif aug_type == 'scale_0.8':
        height, width = image.shape[:2]
        image = cv2.resize(image, (int(width * 0.8), int(height * 0.8)))
    
    return image, aug_type

def augment_images(image_dir, output_dir, output_csv):
    os.makedirs(output_dir, exist_ok=True)
    
    image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg')) and '_aug' not in f]
    
    selected_images = random.sample(image_files, 810)
    
    augmented_data = []
    existing_files = set(os.listdir(output_dir)) 
    
    for image_name in selected_images:
        image_path = os.path.join(image_dir, image_name)
        image = cv2.imread(image_path)
        
        if image is None:
            continue 
    
        aug_image, aug_type = random_augmentation(image)
        
        base_name = os.path.splitext(image_name)[0]
        new_image_name = base_name + '_aug.jpg'
        
        count = 1
        while new_image_name in existing_files:
            new_image_name = base_name + f'_aug{count}.jpg'
            count += 1
        
        new_image_path = os.path.join(output_dir, new_image_name)

        cv2.imwrite(new_image_path, aug_image, [int(cv2.IMWRITE_JPEG_QUALITY), 100])
        
        augmented_data.append([new_image_name, aug_type])
        existing_files.add(new_image_name)  

    df = pd.DataFrame(augmented_data, columns=['new_image_name', 'augmentation_type'])
    df.to_csv(output_csv, index=False)
    print(f"Augmentation complete. New images saved in {output_dir}, and CSV saved as {output_csv}")

image_directory = "/home/axernox/Project_2/Sensitive/" 
output_directory = "/home/axernox/Project_2/Aug_Sensitive/"  
output_csv_file = "augmented_images1.csv"

augment_images(image_directory, output_directory, output_csv_file)

Augmentation complete. New images saved in /home/axernox/Project_2/Aug_Sensitive/, and CSV saved as augmented_images1.csv


In [ ]:
import os
import shutil
import pandas as pd

def separate_sensitive_images(image_dir, csv_path, output_dir):
    df = pd.read_csv(csv_path)
    
    sensitive_df = df[df['model usage'] == 'sensitive']
    
    for folder in ['train', 'test', 'val']:
        os.makedirs(os.path.join(output_dir, folder), exist_ok=True)
    
    for _, row in sensitive_df.iterrows():
        image_name = row['Image_Name']
        split_folder = row['final model']
        
        src_path = os.path.join(image_dir, image_name)
        dest_path = os.path.join(output_dir, split_folder, image_name)
        
        if os.path.exists(src_path):
            shutil.move(src_path, dest_path)
            print(f"Moved {image_name} to {split_folder} folder.")
        else:
            print(f"Image {image_name} not found in source directory.")

    print("Separation complete.")

image_directory = "/home/axernox/Project_2/Sensitive/"
csv_file = "/home/axernox/Project_2/Processed_TB_CXRs_Data.csv"
output_directory = "/home/axernox/Project_2/Sensitive_sep/" 

separate_sensitive_images(image_directory, csv_file, output_directory)

Moved 1.2.840.113564.17216299.20190101084552531300.1003000225002_aug.jpg to test folder.
Moved 2.25.100085223121593679565168768383246568962.jpg to test folder.
Moved 2.25.100094253425368441225245964225739338535_aug1.jpg to test folder.
Moved 2.25.100262136311268540559078515190309455389_aug1.jpg to test folder.
Moved 2.25.101297193828635275908048379644391761168.jpg to test folder.
Moved 2.25.101297193828635275908048379644391761168_aug1.jpg to test folder.
Moved 2.25.101322816590861950430859048376147592743.jpg to test folder.
Moved 2.25.101530826766611761898595192804750256858.jpg to test folder.
Moved 2.25.102589671820934021539688763273077773992.jpg to test folder.
Moved 2.25.102802428853220842799131317284346899670_aug1.jpg to test folder.
Moved 2.25.103235349640221380886091705057947181934.jpg to test folder.
Moved 2.25.104199015472644279225963227657318370599.jpg to test folder.
Moved 2.25.105984524760747072701184633142211113233_aug1.jpg to test folder.
Moved 2.25.10649232942913449259222